In [2]:
import os
import sys
import json
import cv2 as cv
import matplotlib.pyplot as plt
import numpy as np
from tqdm.auto import tqdm
import pickle

parent_dir = "/".join(sys.path[0].split("/")[:-1])
sys.path.append(parent_dir)

data_path = "../data/Cricket-Semantic-Segmentation"

In [4]:
classes = {}
with open(f"{data_path}/classes/classes.json") as handler:
    classes = json.load(handler)
    classes = list(map(lambda obj: {"name":obj["name"], "color": obj["color"]}, classes))

In [5]:
allowed_colors = set(map(lambda cls: cls["color"], classes))
segments = list(filter(lambda name: "__fuse" in name, os.listdir(f"{data_path}/images")))

In [4]:
# from src.data.analyze import findColors

# with tqdm(total=len(segments)) as pbar:
#     for seg in segments:
#         img = cv.imread(f"{data_path}/images/{seg}")
#         available_colors = findColors(img)
#         ok = len(allowed_colors-available_colors) == len(allowed_colors)-len(available_colors)
#         if not ok:
#             print(seg, available_colors)
#         pbar.update(1)

In [64]:
from src.data.analyze import img2ColorMat
from src.data.process import getBoundingBoxesFromSegmentation

labels = list(filter(lambda cls: cls["name"] in ("Wicket", "Ball", "Batsmen"), classes))
boxes_object = []
with tqdm(total=len(segments)) as pbar:
    for seg in segments:
        img = cv.imread(f"{data_path}/images/{seg}")
        box_groups = getBoundingBoxesFromSegmentation(img, labels)
        boxes_object.append({"img_name": seg, "box_groups": box_groups})
        pbar.update(1)

with open("objects.pkl", "wb") as handler:
    pickle.dump(boxes_object, handler)

  0%|          | 0/298 [00:00<?, ?it/s]

In [7]:
objects = []
with open("../data/Cricket-Semantic-Segmentation/objects/objects.pkl", "rb") as handler:
    objects = pickle.load(handler)